## Installation
-  Run this only once

In [ ]:
!pip install gdown
!pip install lightfm
!pip install recommenders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

!pip install numpy==1.22
!pip install LibRecommender


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Restart 

In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [ ]:
TIMESTAMP=None

In [ ]:
!gdown https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6

Downloading...
From: https://drive.google.com/uc?id=1tZlrPeJf7geYfRRTxeRhdPU_Hq5m8tg6
To: /content/Travel_TripAdvisor_v1.zip
100% 155k/155k [00:00<00:00, 83.5MB/s]


In [ ]:
!unzip "Travel_TripAdvisor_v1.zip"  -d  "/content"

Archive:  Travel_TripAdvisor_v1.zip
replace /content/Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install pandas openpyxl

In [ ]:

import pandas as pd

xls = pd.ExcelFile(r"Travel_TripAdvisor_v1/Data_TripAdvisor_v1.xls") 

rating_df = xls.parse(0) #2 is the sheet number+1 thus if the file has only 1 sheet write 0 in paranthesis



In [ ]:
rating_df=rating_df.reset_index(drop=True)

In [ ]:

rating_df.rename(columns = {'ID_USER':'userID'}, inplace = True)
rating_df.rename(columns = {'ID_HOTEL':'itemID'}, inplace = True)
rating_df.rename(columns = {'Rating':'rating'}, inplace = True)

## Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , [other features]** ]


### Dense columns

In [ ]:
dense_col=None

### Sparse columns

In [ ]:
col_sparse= ['USER_STATE', 'USER_TIMEZONE','Trip Type']


### User contexual features other than "user" column

In [ ]:

user_col = ['USER_STATE', 'USER_TIMEZONE','Trip Type']


### User contexual features other than "item" column

In [ ]:
item_col=[]


### Run below cell

In [ ]:
for i in user_col:
  rating_df[i] = rating_df[i].fillna('NA')

In [ ]:
for i in item_col:
  rating_df[i] = rating_df[i].fillna('NA')

In [ ]:
allcols=['userID','itemID','rating']
if user_col is not None:
  for i in user_col:
    allcols.append(i)
if item_col is not None:
  for i in item_col:
    allcols.append(i)

In [ ]:
listcols = []
[listcols.append(x) for x in allcols if x not in listcols]

[None, None, None, None, None, None]

In [ ]:
listcols

['userID', 'itemID', 'rating', 'USER_STATE', 'USER_TIMEZONE', 'Trip Type']

In [ ]:
rating_df=rating_df[listcols]

##### Code Setup

In [ ]:
import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation
# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k 
from lightfm.evaluation import recall_at_k 
# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)
from lightfm import cross_validation
# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k 
from lightfm.evaluation import recall_at_k 

#### LightFM code

In [ ]:

def runlightFM(rating_df,user_col,item_col,test_percentage=.2,no_components=50,seed=42,k=10,lr=0.1,item_alpha = 1e-6,user_alpha = 1e-6,epochs=100,threads=32):
    user_features =list()
    item_features=list()
    if len(user_col)>0:
      user_features=list( rating_df[user_col])
    if len(item_col)>0:
        item_features=list( rating_df[item_col])
    user_features=iter(user_features)
    item_features=iter(item_features)
    
    dataset = Dataset()
    dataset.fit(users=rating_df['userID'], 
            items=rating_df['itemID'],
             user_features=user_features,
            item_features=item_features
            )
    user_features = dataset.build_user_features((x, y) for x,y in zip(rating_df.userID,user_features))
    item_features = dataset.build_item_features((x, y) for x,y in zip(rating_df.itemID,item_features))
    # quick check to determine the number of unique users and items in the data
    num_users, num_items = dataset.interactions_shape()
    print(f'Num users: {num_users}, num_items: {num_items}.')
    interactions, weights = dataset.build_interactions(rating_df[['userID','itemID','rating']].values)


    train_interactions, test_interactions = cross_validation.random_train_test_split(
    interactions, test_percentage=test_percentage,
    random_state=np.random.RandomState(seed))
    model = LightFM(loss='warp', no_components=no_components, 
                 learning_rate=lr, 
                 item_alpha=item_alpha,
                 user_alpha=user_alpha,
                 random_state=np.random.RandomState(seed)
                )
    model.fit(interactions=train_interactions,
           user_features=user_features,
           item_features=item_features,
           epochs=epochs
           )
    uids, iids, interaction_data = cross_validation._shuffle(
    interactions.row, 
    interactions.col, 
    interactions.data, 
    random_state=np.random.RandomState(seed)
    )

    uid_map, ufeature_map, iid_map, ifeature_map = dataset.mapping()
    cutoff = int((1.0 - test_percentage) * len(uids))
    test_idx = slice(cutoff, None)
    test_df = prepare_test_df(test_idx, uids, iids, uid_map, iid_map, weights)
    all_predictions = prepare_all_predictions(rating_df, uid_map, iid_map, 
                                              interactions=train_interactions,
                                              model=model, 
                                              user_features=user_features,
                                              item_features=item_features,
                                              num_threads=threads)
    top_k_items = (
            all_predictions.sort_values(["userID","prediction" ], ascending=[True, False])
            .groupby("userID", as_index=False)
            .head(k)
            .reset_index(drop=True)
        )
 
    top_k_items = top_k_items[["userID","itemID"]]
    print("Train precision: %.5f" % precision_at_k(model, train_interactions, k=10, user_features=user_features,item_features=item_features,).mean())
    print("Test precision: %.5f" % precision_at_k(model, test_interactions, k=10,user_features=user_features,item_features=item_features,).mean())
    return model,    top_k_items


#### Lib recommender code

In [ ]:
# Evaluation metrics
import libreco
import tensorflow as tf
from libreco.data import DatasetFeat, DataInfo
from libreco.data import  split_by_ratio
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats
import torch
from libreco.algorithms import (
    FM,
    
)

import time
import pandas as pd

metrics=[
            "loss",
            "balanced_accuracy",
            "roc_auc",
            "pr_auc",
            "precision",
            "recall",
            "map",
            "ndcg",
        ]

# Call Librecommender API
# datasettype ="pure" or "feat"
def run(rating_df,user_col, item_col, col_sparse, dense_col,datasettype="pure", algo=["als"],hyperparam={"als":{ "embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}}, neg_samples=True):
    train_data, test_data, data_info=None,None,None
    tf.compat.v1.reset_default_graph()
    train_data, test_data, data_info = preprocessingFeature(rating_df,user_col, item_col, col_sparse, dense_col, neg_samples) 
    models = dict()
    evals = dict()
    if(neg_samples):
      train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
      test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    for i in algo:
        if i == 'FM':
            
            als, eval, algo_data_info = FM(train_data, test_data, data_info,hyperparam["FM"])
            models['FM'] = als
            evals['FM'] = eval
            print("Evaluation Results for FM:")
            print(evals['FM'])
        
       
          
    return models, evals

# data preprocessing
def preprocessingFeature(rating_df,user_col, item_col, col_sparse, dense_col, neg_samples=True):
    data = rating_df.copy()

    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetFeat.build_trainset(
        train, user_col, item_col, col_sparse, dense_col,
        shuffle=False,
    )
    test_data = DatasetFeat.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info


# Recommend items to user
def recommend(model, userid, k=10):
    return model.recommend_user(user=userid, n_rec=k)

# Find similar items
def similarItems(model, itemids, k=10):
    return model.search_knn_items(item=itemids, k=k)


# Model : Wide and deep
def FM(train_data, test_data, data_info,hyperparam={ "loss_type":"cross_entropy","embed_size":16,
        "n_epochs":10,
        "lr":1e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":256,
        "num_neg":1,
        "use_bn":False,
        "dropout_rate":None,
        "tf_sess_config":None,}):
  
    
    model = libreco.algorithms.FM(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"] ,
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],  
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        use_bn=hyperparam["use_bn"],
        dropout_rate=hyperparam["dropout_rate"],
      
        tf_sess_config=hyperparam["tf_sess_config"],
    )
    model.fit(
        train_data,
        verbose=2,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=model, data=test_data, metrics=metrics)
    return model, result, data_info

    


##  LightFM

In [ ]:

model,pred=runlightFM(rating_df,user_col,item_col)


#### Recommendations to users

In [ ]:
pred.head(10)

## Libreco FM

In [ ]:
recofm_df=rating_df

In [ ]:

recofm_df.rename(columns = {'userID':'user'}, inplace = True)
recofm_df.rename(columns = {'itemID':'item'}, inplace = True)
recofm_df.rename(columns = {'label':'label'}, inplace = True)

In [ ]:
recofm_df['label']=1

In [ ]:

models, evals = run(recofm_df,user_col, item_col, col_sparse, dense_col, algo=["FM"], hyperparam={"FM":{"loss_type":"cross_entropy","embed_size":64,
        "n_epochs":50,
        "lr":1e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":128,
        "num_neg":2,
        "use_bn":False,
        "dropout_rate":None,
        "tf_sess_config":None,} },neg_samples=True)


random neg item sampling elapsed: 0.005s
random neg item sampling elapsed: 0.000s
random neg item sampling elapsed: 0.009s
random neg item sampling elapsed: 0.001s
total params: 199,296 | embedding params: 199,227 | network params: 69
Training start time: 2023-02-08 07:50:15


train: 100%|██████████| 129/129 [00:02<00:00, 46.96it/s] 


Epoch 1 elapsed: 2.756s
	 train_loss: 0.6935


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 38.29it/s]


	 eval log_loss: 0.6934
	 eval balanced accuracy: 0.4927
	 eval roc_auc: 0.5006
	 eval pr_auc: 0.4985


...

	 eval log_loss: 0.6935
	 eval balanced accuracy: 0.5308
	 eval roc_auc: 0.5321
	 eval pr_auc: 0.5467


eval_listwise: 100%|██████████| 23/23 [00:00<00:00, 190.40it/s]


	 eval precision@10: 0.0019
	 eval recall@10: 0.0170
	 eval map@10: 0.0066
	 eval ndcg@10: 0.0096


eval_listwise: 100%|██████████| 103/103 [00:00<00:00, 525.93it/s]


Evaluation Results for FM:
{'loss': 0.6934993142370317, 'balanced_accuracy': 0.530829596412556, 'roc_auc': 0.5321247410967443, 'pr_auc': 0.5466939274092928, 'precision': 0.0019464720194647203, 'recall': 0.0170316301703163, 'map': 0.006584791256324102, 'ndcg': 0.009566812041935375}


#### Recommendations to users

In [ ]:
model=models['FM']

In [ ]:

results= recommend(model,65457, 10)
print(results)

{65457: array([  73821,   74362,   74476,   79445,  125845,  100602,   84031,
        878115, 1225375,   84654])}
